# N-Gram-Based Text Categorization
http://odur.let.rug.nl/~vannoord/TextCat/textcat.pdf

William B. Cavnar and John M. Trenkle (1994)

#### Implementations

* https://github.com/FGRibreau/node-language-detect

Can be used for language classification, or topic classification

### Zipf’s Law
The nth most common word in a human language text occurs with a frequency inversely proportional to n.

https://www.youtube.com/watch?v=fCn8zs912OE <br>
https://en.wikipedia.org/wiki/Zipf%27s_law